In [1]:
!pip install deepface ultralytics faiss-cpu pandas

In [2]:
from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import time
import random

import numpy as np
import faiss

In [3]:
model_name = "Facenet" 
detector_backend = "opencv"
num_dimensions = 128 # Facenet output

In [ ]:
representations = []

# Inisialisasi DataFrame Pandas untuk menyimpan nama orang dan embedding
df = pd.DataFrame(columns=["name", "embedding", "posisi"])

# Dictionary untuk mapping file_name ke nama orang
file_to_name = {
    "img5.jpg" : "Angelina Jolie",
    "img6.jpg" : "Angelina Jolie",
    "img14.jpg" : "Mark Zuckerberg",
    "img15.jpg" : "Mark Zuckerberg",
    "img17.jpg" : "Jack Dorsey",
    "img54.jpg" : "Jennifer Aniston",
    "img56.jpg" : "Jennifer Aniston",
    "img62.jpg" : "Jack Dorsey",
    "heydar.jpg" : "Heydar",
    "ica.jpg" : "Ica",
    "nola.jpg" : "Nola",
    "indah.jpg" : "Indah",
    "irsan.jpg" : "Irsan",
    "zeyad.jpg" : "Zeyad"
}
name_to_position = {
    "Angelina Jolie" : "Oscar Actress",
    "Mark Zuckerberg" : "CEO Meta",
    "Jack Dorsey" : "CEO Square",
    "Jennifer Aniston" : "Friends serial actress",
    "Heydar" : "walkot",
    "Ica" : "Engineer",
    "Nola" : "CEO rokan hulu",
    "Indah" : "Dirut elnusa",
    "Irsan" : "bupati sleman",
    "Zeyad" : "raja arab"
}
for r, d, files in os.walk("../Belajar-DeepFace"):
    for file in files:
        if ".jpg" in file:
            exact_file = f"{r}/{file}"
            objs = DeepFace.represent(
                img_path=exact_file,
                model_name=model_name,
                detector_backend=detector_backend
            )
            
            # Dapatkan nama orang dari dictionary
            name = file_to_name.get(file, "Unknown")  # Jika file tidak ada dalam dictionary, beri nama 'Unknown'
            posisi = name_to_position.get(name, "unknown position")
            for obj in objs:
                embedding = obj["embedding"]
                representations.append([file, embedding])
                
                # Buat DataFrame baru dari nama dan embedding yang dihasilkan
                new_row = pd.DataFrame({"name": [name], "embedding": [embedding], "posisi": [posisi]})
                
                # Gabungkan DataFrame baru ke DataFrame utama menggunakan pd.concat()
                df = pd.concat([df, new_row], ignore_index=True)

In [ ]:
# number of dimension
for obj in objs:
    print(len(obj["embedding"]))

In [ ]:
print(df)

In [ ]:
# Konversi embedding ke numpy array untuk FAISS
embeddings = df['embedding'].tolist()  # Ambil embedding dari DataFrame
embeddings = np.array(embeddings, dtype="f")  # Konversi ke numpy array

In [ ]:
# Inisialisasi FAISS
index = faiss.IndexFlatL2(num_dimensions)

In [ ]:
# Tambahkan embedding ke FAISS
index.add(embeddings)

In [ ]:
# Simpan DataFrame ke CSV jika diperlukan
df.to_csv("face_embeddings.csv", index=False)

In [ ]:
print(f"deepface unit test items have {len(representations)} images")

In [ ]:
embeddings = []
for key, value in representations:
    embeddings.append(value)

In [ ]:
# euclidean
tic = time.time()
index = faiss.IndexFlatL2(num_dimensions)
toc = time.time()
print(f"faiss instance is initialized in {toc-tic} seconds")

In [ ]:
tic = time.time()
index.add(np.array(embeddings, dtype = "f"))
toc = time.time()
print(f"{len(embeddings)} vectors stored in faiss intance in {toc-tic} seconds")

In [ ]:
# --- Tambahkan Webcam Input dengan Capturing Otomatis Tiap 10 Detik ---

# Inisialisasi webcam
cap = cv2.VideoCapture(1)

img_counter = 0
capture_interval = 10  # Interval capture dalam detik
start_time = time.time()  # Waktu awal

while True:
    # Baca frame dari webcam
    ret, frame = cap.read()

    # Tampilkan frame dari webcam
    cv2.imshow('Webcam', frame)

    # Cek waktu, jika sudah lebih dari 10 detik, capture gambar
    current_time = time.time()
    if current_time - start_time >= capture_interval:
        img_counter += 1

        # Alih-alih menyimpan frame yang di-capture, kita langsung memproses frame di memori
        print(f"Gambar ke-{img_counter} di-capture dan sedang diproses...")

        # Proses gambar target dari webcam
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Konversi ke RGB
        target_embedding = DeepFace.represent(
            img_path=rgb_frame,  # Gunakan frame RGB langsung dari memori
            model_name="Facenet",
            detector_backend="opencv",
            enforce_detection=False  # Allow processing even if no face detected
        )[0]["embedding"]

        # Konversi embedding ke array numpy dan expand dimensinya
        target_embedding = np.array(target_embedding, dtype="f")
        target_embedding = np.expand_dims(target_embedding, axis=0)

        # Cari embedding terdekat di FAISS
        k = 1  # Jumlah neighbor terdekat yang ingin dicari
        distances, neighbours = index.search(target_embedding, k)

        # Tampilkan hasil pencarian dari DataFrame
        for idx, neighbour in enumerate(neighbours[0]):
            if neighbour < len(df):
                neighbour_name = df.iloc[neighbour]['name']
                neighbour_position = df.iloc[neighbour]['posisi']
                neighbour_img_path = f"../Belajar-DeepFace/{representations[neighbour][0]}"
                
                # Ambil wajah neighbor dari gambar
                neighbour_img = DeepFace.extract_faces(
                    img_path=neighbour_img_path,
                    detector_backend="opencv"
                )[0]["face"]
                
                # Visualisasikan gambar target dan neighbor
                print(f"{idx+1}th closest match: {neighbour_name}")
                
                fig = plt.figure(figsize=(7, 7))
                fig.add_subplot(1, 2, 1)    
                plt.imshow(rgb_frame)  # Gunakan gambar RGB untuk ditampilkan
                plt.axis("off")
                fig.add_subplot(1, 2, 2)
                plt.imshow(neighbour_img)
                plt.axis("off")
                plt.show()

            else:
                print(f"Index {neighbour} out of range for DataFrame")

        print(f"Hallo {neighbour_name} {neighbour_position}, selamat datang")

        # Reset waktu awal setelah capture
        start_time = current_time

    # Tekan ESC untuk keluar
    if cv2.waitKey(1) & 0xFF == 27:
        print("Escape hit, closing...")
        break

# Lepaskan resource webcam dan tutup window
cap.release()
cv2.destroyAllWindows()
